<a href="https://colab.research.google.com/github/dkshjn/Technoculture/blob/main/Adapter_Function_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install peft
!pip install git+https://github.com/huggingface/transformers
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 33.2 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-49xo40lx
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-49xo40lx
  Resolved https://github.com/huggingface/transformers to commit d628664688b05cabdd69f4e7e295bc4aee0a8d31
  Installing build dependencies ... 

In [2]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')
!huggingface-cli login --token $hf_token

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
from datasets import load_dataset

dataset = load_dataset("Technoculture/chatdoctor-200k-stripped-embedded", split="s1")

Generating s1 split:   0%|          | 0/82964 [00:00<?, ? examples/s]

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig

class CustomModel:
    def __init__(self, model_id, peft_model_id):
        self.model = AutoModelForCausalLM.from_pretrained(model_id)
        self.model.load_adapter(peft_model_id)
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)

    def __call__(self, prompt, max_length=200, num_return_sequences=1):
        inputs = self.tokenizer(prompt, return_tensors="pt")
        outputs = self.model.generate(input_ids=inputs["input_ids"], max_length=max_length, num_return_sequences=num_return_sequences)
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response

In [5]:
model_id = "Technoculture/MT7Bi-sft"
peft_model_id = "Technoculture/MT7Bi-alpha-dpo-v0.2"

# Create an instance of the custom model
model = CustomModel(model_id, peft_model_id)

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.23k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/344 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
model("Why are injections inserted into veins?")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'Why are injections inserted into veins?\nWhy are injections given in the vein? \n\n### 1. REF:\n\nThe veins are larger than the arteries and are more superficial.\n\n2. REF:\n\nThe veins are more superficial and are less likely to be damaged by the needle.\n\n3. REF:\n\nThe veins are more superficial and are less likely to be damaged by the needle.\n\n### 4. REF:\n\nThe veins are more superficial and are less likely to be damaged by the needle.\n\n### 5. REF:\n\nThe veins are more superficial and are less likely to be damaged by the needle.\n\n### 6. REF:\n\nThe veins are more superficial and are less'

In [7]:
def compute_similarity(target_embedding, embeddings):
    similarities = cosine_similarity([target_embedding], embeddings)
    return similarities[0]

# Select a random target question from the dataset
import random
random.seed(10)
target_question_index = random.randint(0, len(dataset)-1)
target_question = dataset["input"][target_question_index]
target_in_embedding = dataset["embeddings_input"][target_question_index]

In [8]:
target_question_index

74894

In [9]:
target_question

'My 18 yr old son has had blisters on his throat and now spreading to the roof of his mouth he also has a yellowish color on his younger that does not brush away when he eats or drinks certain things he says it burns or hurts ,he does usually use smokeless tobacco but has cut back drastically and nothing is helping what could this be Andis he contagious?'

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
embeddings = dataset["embeddings_input"]
similarities = compute_similarity(target_in_embedding, embeddings)

In [15]:
df = dataset.to_pandas()
df['similarity'] = similarities

df_sorted = df.sort_values(by='similarity', ascending=True)
df_filtered = df_sorted[df_sorted['input'].apply(lambda x: len(str(x)) > 100)]

# Get the top 10 unrelated questions from the 'input' column
df_filtered_top10 = df_filtered.head(10)
top_unrelated_questions = df_filtered.head(10)['input'].tolist()
top_unrelated_questions

['How does the brain work? I know all thought is based on association, which is formed in a grid of neurons and synapses - but when I (my ego) actively query myself for information, how does the brain transmit the query and how does it collect the memory? Do we know this yet? Please, use all the terminology you have.',
 'Have been recently diagnost with endometrial cancer. Been told I have to be cut from the naval down to pelvis.  Isnt there a less invasive procedure that is just as good. Doing a Total abdominal hysterectomy & bilateral Salpingo-oophorectomy and staging.',
 'my wife had a stroke on5th may2010.was operated and skull bone of the right hemisphere removed.she has continued to be unconscious since then.how long will she continue as such?',
 'I looked online to see if it is possible to get pregnant after your ovulation cycle. All i got was confusing answers. I have a regular 28 day cycle and ovulate 14 after my period. I had sex 5 days after my ovulation cycle, is it possibl

In [17]:
df_filtered_top10

,input,instruction,output,embeddings_input,embeddings_output,similarity
5481,How does the brain work? I know all thought is...,You are a doctor. Answer the medical questions...,Hi Lux. Thanks for your query and sing Chat Do...,"[-0.044524941593408585, 0.08839308470487595, -...","[-0.3996073305606842, 0.2814745306968689, 0.10...",0.399252
7194,Have been recently diagnost with endometrial c...,You are a doctor. Answer the medical questions...,"Hi, dairy have gone through your question. I c...","[0.591637134552002, -0.038249578326940536, 0.1...","[0.33395740389823914, -0.2914787232875824, -0....",0.402220
13277,my wife had a stroke on5th may2010.was operate...,You are a doctor. Answer the medical questions...,I am afraid your wife has suffered a very big ...,"[-0.3325180411338806, -0.14967140555381775, 0....","[-0.21583226323127747, -0.06710463017225266, 0...",0.411491
3899,I looked online to see if it is possible to ge...,You are a doctor. Answer the medical questions...,"Hi, Welcome to Chat Doctor. I have read the qu...","[0.024531692266464233, 0.396260142326355, -0.5...","[0.06398504972457886, 0.5252920985221863, -0.3...",0.411517
69905,Is there a difference in a total abdominal hys...,"If you are a doctor, please answer the medical...",Hi. Thanks for a nice question and appreciatio...,"[-0.15490856766700745, 0.3571265935897827, 0.2...","[0.07165086269378662, 0.06117568537592888, 0.2...",0.412315
41700,"hello dr. iam 27 yrs old my height 5 4 , weigh...","If you are a doctor, please answer the medical...","Welcome to ChatDoctorRead your prob, your endo...","[-0.40338441729545593, -0.0047707026824355125,...","[0.4827222228050232, 0.2608940005302429, -0.00...",0.415314
38956,i have a broken finger which i had surgery on ...,You are a doctor. Answer the medical questions...,"Hello, It has to be kept for two to three week...","[-0.4292530417442322, -0.46730539202690125, 0....","[-0.40246981382369995, 0.04907269775867462, 0....",0.417087
4943,How long before I can exercise after an open c...,You are a doctor. Answer the medical questions...,Hi and welcome to Chat Doctor. Thank you for y...,"[0.09118236601352692, -0.29786598682403564, 0....","[-0.05712539330124855, 0.24290399253368378, 0....",0.419805
39580,I suffered a comminuted fracture of the right ...,You are a doctor. Answer the medical questions...,"Hello, Newer techniques like Illizarrow may be...","[-0.45141154527664185, -0.38623762130737305, 0...","[-0.5285519361495972, -0.3605390787124634, 0.3...",0.422569
58177,i had aortic valve replacement and ascending a...,"If you are a doctor, please answer the medical...","Dear Patrick, Heart rate until 100-105 will be...","[-0.9276523590087891, -0.13429448008537292, 0....","[-0.19187739491462708, -0.08047466725111008, 0...",0.424040


In [19]:
from tqdm import tqdm
responses = []
for input in tqdm(top_unrelated_questions):
    response = model(input)
    responses.append(response)

  0%|          | 0/10 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
 10%|█         | 1/10 [01:32<13:50, 92.31s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
 20%|██        | 2/10 [03:04<12:16, 92.09s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
 30%|███       | 3/10 [04:17<09:43, 83.37s/it]The attention mask and the pad token id were not set. As a consequence, you may obs

In [20]:
responses

['How does the brain work? I know all thought is based on association, which is formed in a grid of neurons and synapses - but when I (my ego) actively query myself for information, how does the brain transmit the query and how does it collect the memory? Do we know this yet? Please, use all the terminology you have.\n\nThe brain is a complex system, and we are only beginning to understand how it works. The brain is composed of billions of neurons, which are the basic units of the nervous system. Neurons communicate with each other through synapses, which are specialized junctions between neurons. When a neuron receives an electrical signal, it sends an electrical impulse down its axon. This impulse is transmitted to the next neuron through the synapse. The receiving neuron then sends an electronic signal back to the sending neuron, which in',
 'Have been recently diagnost with endometrial cancer. Been told I have to be cut from the naval down to pelvis.  Isnt there a less invasive pro